# Подготовка данных

In [11]:
# Импортируем основные модули и функции
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import train_test_split

# Игнорируем возможные предупреждения в силу ограничений на количество итераций для моделей
import warnings
warnings.filterwarnings('ignore', category=Warning)

In [12]:
# Читаем данные
data = pd.read_csv('data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Разбиваем данные на матрицу наблюдей 'X' и целевой признак 'y'
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [14]:
# Разбиваем данные на тренировочную и валидационную выборки (в соотношении 70/30)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# Повторно разбиваем данные но уже на валидационную и тестовую выборки (в соотношении 50/50)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=42)

In [15]:
# Выводим соотношение категорий целевого признака для каждой выборки 
print('Train:\n', y_train.value_counts(normalize=True), sep='')
print('\n')
print('Valid:\n', y_valid.value_counts(normalize=True), sep='')
print('\n')
print('Test:\n', y_test.value_counts(normalize=True), sep='')

Train:
Activity
1    0.542095
0    0.457905
Name: proportion, dtype: float64


Valid:
Activity
1    0.536412
0    0.463588
Name: proportion, dtype: float64


Test:
Activity
1    0.548845
0    0.451155
Name: proportion, dtype: float64


In [16]:
# Выводим размерности каждой выборки
print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))
print('Test shape: {}'.format(X_test.shape))

Train shape: (2625, 1776)
Valid shape: (563, 1776)
Test shape: (563, 1776)


In [17]:
# Создаём модель логистической регресии с параметрами по умолчанию (учитывая ограничения на итерации)
log_reg = linear_model.LogisticRegression(max_iter=50, random_state=42)

# Обучаем модель
log_reg.fit(X_train, y_train)
# Делаем предсказание на тестовой выборке и выводим значение метрики
y_test_pred = log_reg.predict(X_test)
print('F1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовой выборке: 0.76


In [18]:
# Создаём модель случайного леса с параметрами по умолчанию
fr = ensemble.RandomForestClassifier(random_state=42)

# Обучаем модель
fr.fit(X_train, y_train)
# Делаем предсказание на тестовой выборке и выводим значение метрики
y_test_pred = fr.predict(X_test)
print('F1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовой выборке: 0.79


# Базовая оптимизация

## GridSearchCV

### **Логистическая регрессия**

In [19]:
# Импортируем класс 'GridSearchCV'
from sklearn.model_selection import GridSearchCV

# Указываем искомые гиперпараметры в виде списка словарей 'param_grid'
param_grid = [
              {'penalty': ['l2', None], # Тип регуляризации
              'solver': ['lbfgs', 'sag'], # Алгоритм оптимизации
              'C': [0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # Уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
              'C': [0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

# Вызваем класс `GridSearchCV`
log_grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), # Передаём нашу модель (`LogisticRegression`)
    param_grid=param_grid, # Сетку искомых параметров
    cv=5, # Число фолдов для кросс-валидации
    n_jobs = -1 # `n_jobs = -1`, чтобы использовать все доступные ядра для расчётов
)  

# Обучаем объект `GridSearchCV` на валидационной выборке
log_grid_search.fit(X_valid, y_valid) 
# Делаем предсказание на валидационной выборке
y_valid_pred = log_grid_search.predict(X_valid)
# Выводим значения метрики и гиперпараметров
print('F1-score на валидационной наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
print("Наилучшие значения гиперпараметров: {}".format(log_grid_search.best_params_))

F1-score на валидационной наборе: 0.83
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


In [20]:
# Создаём и обучаем модель с полученными гиперпараметрами
log_reg = linear_model.LogisticRegression(
    C=0.3,
    penalty='l1',
    solver='saga',
    random_state=42,
    max_iter=50
)
log_reg.fit(X_train, y_train)

# Делаем предсказание на тестовой выборке и выводим значение метрики
y_test_pred = log_reg.predict(X_test)
print('F1-score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовом наборе: 0.78


### **Случайный лес**

In [21]:
# Зададим сетку гиперпараметров для 'RandomForestClassifier' 
param_grid = {'n_estimators': list(range(80, 200, 30)), # Количество деревьев в лесу
              'min_samples_leaf': [5], # Минимальное количество объектов в листе
              'max_depth': list(range(3, 15, 3)) # Максимальная глубина дерева
}

# Проводим шаги, аналогичные предыдущей модели
rf_grid_search = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5,
    n_jobs = -1
)

rf_grid_search.fit(X_valid, y_valid) 
y_valid_pred = rf_grid_search.predict(X_valid)
print('F1-score на валидационной наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
print("Наилучшие значения гиперпараметров: {}".format(rf_grid_search.best_params_))

F1-score на валидационной наборе: 0.88
Наилучшие значения гиперпараметров: {'max_depth': 6, 'min_samples_leaf': 5, 'n_estimators': 80}


In [22]:
# Создаём и обучаем модель с полученными гиперпараметрами
rf = ensemble.RandomForestClassifier(max_depth=6, min_samples_leaf=5, n_estimators=80, random_state=42)
rf.fit(X_train, y_train)

# Делаем предсказание на тестовой выборке и выводим значение метрики
y_test_pred = rf.predict(X_test)
print('F1-score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовом наборе: 0.78


## RandomizedSearchCV

### **Логистическая регрессия**

In [23]:
# Импортируем класс 'RandomizedSearchCV'
from sklearn.model_selection import RandomizedSearchCV

# Указываем искомые гиперпараметры в виде списка словарей 'param_distributions'
param_distributions = [
                       {'penalty': ['l2', None] , # Тип регуляризации
                       'solver': ['lbfgs', 'sag'], # Алгоритм оптимизации
                       'C': list(np.linspace(0.01, 1, 10, dtype=float))}, # Уровень силы регурялизации
              
                       {'penalty': ['l1', 'l2'] ,
                       'solver': ['liblinear', 'saga'],
                       'C': list(np.linspace(0.01, 1, 10, dtype=float))}
]

# Вызваем класс `RandomizedSearchCV`
log_random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), # Передаём нашу модель (`LogisticRegression`)
    param_distributions=param_distributions, # Сетку искомых параметров
    cv=5, # Число фолдов для кросс-валидации
    n_iter = 25, # Количество комбинаций на расчёт
    n_jobs = -1, # `n_jobs = -1`, чтобы использовать все доступные ядра для расчётов
    random_state=6 # Фиксируем результаты для каждого запуска
)  
 
# Обучаем объект `RandomizedSearchCV` на валидационной выборке
log_random_search.fit(X_valid, y_valid) 
# Делаем предсказание на валидационной выборке
y_valid_pred = log_random_search.predict(X_valid)
# Выводим значения метрики и гиперпараметров
print('F1-score на валидационной наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
print("Наилучшие значения гиперпараметров: {}".format(log_random_search.best_params_))

F1-score на валидационной наборе: 0.82
Наилучшие значения гиперпараметров: {'solver': 'liblinear', 'penalty': 'l1', 'C': np.float64(0.23)}


In [24]:
# Создаём и обучаем модель с полученными гиперпараметрами
log_reg = linear_model.LogisticRegression(
    solver='liblinear',
    penalty='l1',
    C=0.23,
    random_state=42,
    max_iter=50
)
log_reg.fit(X_train, y_train)

# Делаем предсказание на тестовой выборке и выводим значение метрики
y_test_pred = log_reg.predict(X_test)
print('F1-score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовом наборе: 0.79


### **Случайный лес**

In [16]:
# Зададим сетку гиперпараметров для 'RandomForestClassifier' 
param_distributions = {'n_estimators': list(range(80, 200, 30)), # Количество деревьев в лесу
                       'min_samples_leaf': [5], # Минимальное количество объектов в листе
                       'max_depth': list(range(3, 15, 3)) # Максимальная глубина дерева
}

# Проводим шаги, аналогичные предыдущей модели
rf_random_search = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 25,
    n_jobs = -1,
    random_state=6
)

rf_random_search.fit(X_valid, y_valid) 
y_valid_pred = rf_random_search.predict(X_valid)
print('F1-score на валидационной наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
print("Наилучшие значения гиперпараметров: {}".format(rf_random_search.best_params_))

F1-score на валидационной наборе: 0.88
Наилучшие значения гиперпараметров: {'n_estimators': 80, 'min_samples_leaf': 5, 'max_depth': 6}


In [17]:
# Создаём и обучаем модель с полученными гиперпараметрами
rf = ensemble.RandomForestClassifier(n_estimators=80, min_samples_leaf=5, max_depth=6, random_state=42)
rf.fit(X_train, y_train)

# Делаем предсказание на тестовой выборке и выводим значение метрики
y_test_pred = rf.predict(X_test)
print('F1-score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовом наборе: 0.78


# Продвинутая оптимизация

## Hyperopt

### **Логистическая регрессия**

In [25]:
# Проводим импорт необходимых модулей и функций
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров

In [26]:
# Зададим пространство поиска гиперпараметров
space= {'penalty': hp.choice('penalty', ['l1', 'l2']) ,
        'solver': hp.choice('solver', ['liblinear', 'saga']),
        'C': hp.uniform('C', 0.01, 1)
}

# Зафиксируем random_state
random_state = 42

# Функция 'log_hyperopt' получает комбинацию гиперпараметров в 'params'
def log_hyperopt(params, cv=5, X=X_valid, y=y_valid, random_state=random_state):
       params = {'penalty': params['penalty'],
                 'solver': params['solver'],
                 'C': float(params['C'])
}
       # Используем эту комбинацию для построения модели
       model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=50)
       # Обучаем модель
       model.fit(X, y)
       # Применяем `cross validation` с тем же количеством фолдов
       score = cross_val_score(model, X, y, cv=cv, scoring='f1', n_jobs=-1).mean()
       # Возвращаем метрику, которую необходимо минимизировать, так же поэтому ставим знак минус
       return -score

# Начинаем подбор гиперпараметров
best=fmin(log_hyperopt, # Наша функция 
          space=space, # Пространство гиперпараметров
          algo=tpe.suggest, # Алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=25, # Максимальное количество итераций
          rstate=np.random.default_rng(random_state) # Фиксируем 'random_state' для повторяемости результата
)
# Выводим значения найденных гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 25/25 [00:11<00:00,  2.11trial/s, best loss: -0.7592942658386537]
Наилучшие значения гиперпараметров {'C': np.float64(0.30302818618198823), 'penalty': np.int64(0), 'solver': np.int64(1)}


In [27]:
# Строим модель и рассчитываем точность для тестовой выборки
log_model = linear_model.LogisticRegression(
    C=float(best['C']),
    penalty='l1',
    solver='saga',
    random_state=42,
    max_iter=50
)

log_model.fit(X_train, y_train)
y_test_pred = log_model.predict(X_test)
print('F1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовой выборке: 0.78


### **Случайный лес**

In [28]:
# Все шаги описанные ниже аналогичны и для предыдущей модели, за исключением искомых параметров для модели
space={'n_estimators': hp.quniform('n_estimators', 80, 200, 30),
       'min_samples_leaf' : hp.choice('min_samples_leaf', [5]),
       'max_depth': hp.quniform('max_depth', 3, 15, 3)
}

random_state = 42

def rf_hyperopt(params, cv=5, X=X_valid, y=y_valid, random_state=random_state):
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
              'min_samples_leaf': params['min_samples_leaf']
}
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    return -score

best=fmin(rf_hyperopt,
          space=space,
          algo=tpe.suggest,
          max_evals=25,
          rstate=np.random.default_rng(random_state)
)
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 25/25 [00:12<00:00,  1.97trial/s, best loss: -0.7399692507533627]
Наилучшие значения гиперпараметров {'max_depth': np.float64(12.0), 'min_samples_leaf': np.int64(0), 'n_estimators': np.float64(90.0)}


In [29]:
rf = ensemble.RandomForestClassifier(
    max_depth=int(best['max_depth']),
    min_samples_leaf=5,
    n_estimators=int(best['n_estimators']),
    random_state=42
)

rf.fit(X_train, y_train)
y_test_pred = rf.predict(X_test)
print('F1-score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовом наборе: 0.80


## Optuna

### **Логистическая регрессия**

In [30]:
# Импортируем необходимый модуль 
import optuna

In [31]:
def log_optuna(trial):
    # Задаём пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    C = trial.suggest_float('C', 0.01, 1)
    cv=5
    
    # Создаём модель
    model = linear_model.LogisticRegression(penalty=penalty,
                                            solver=solver,
                                            C=C,
                                            random_state=42
)
    # Обучаем модель
    model.fit(X_valid, y_valid)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    return score

# Создаём объект исследования
# Напрямую указываем, что нам необходимо максимизировать метрику (direction='maximize')
study = optuna.create_study(study_name="LogisticRegressionr", direction="maximize")

# Ищем лучшую комбинацию гиперпараметров ('n_trials' раз)
study.optimize(log_optuna, n_trials=25)

[I 2024-09-13 00:26:31,426] A new study created in memory with name: LogisticRegressionr
[I 2024-09-13 00:26:32,007] Trial 0 finished with value: 0.7762601144129195 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.8255370166285899}. Best is trial 0 with value: 0.7762601144129195.
[I 2024-09-13 00:26:35,929] Trial 1 finished with value: 0.7872337094313107 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.37295121186822944}. Best is trial 1 with value: 0.7872337094313107.
[I 2024-09-13 00:26:38,875] Trial 2 finished with value: 0.778124187159875 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.8920568871160849}. Best is trial 1 with value: 0.7872337094313107.
[I 2024-09-13 00:26:41,594] Trial 3 finished with value: 0.784470703591708 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.10773088132289133}. Best is trial 1 with value: 0.7872337094313107.
[I 2024-09-13 00:26:44,356] Trial 4 finished with value: 0.7778286111586444 and parameters: {'pena

In [32]:
# Выводим результаты на валидационной выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("F1-score на валидационном наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'penalty': 'l1', 'solver': 'saga', 'C': 0.23298087786540236}
F1-score на валидационном наборе: 0.79


In [33]:
# Рассчитываем точность для тестовой выборки
log_model = linear_model.LogisticRegression(**study.best_params, random_state=random_state, max_iter=50)

log_model.fit(X_train, y_train)
y_test_pred = log_model.predict(X_test)
print('F1-score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовой выборке: 0.78


### **Случайный лес**

In [34]:
# Все шаги описанные ниже аналогичны и для предыдущей модели, за исключением искомых параметров для модели
def rf_optuna(trial):
  n_estimators = trial.suggest_int('n_estimators', 80, 200, 30)
  min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [5])
  max_depth = trial.suggest_int('max_depth', 3, 15, 3)
  cv=5

  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=42
)
  
  model.fit(X_valid, y_valid)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
  return score

study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
study.optimize(rf_optuna, n_trials=25)

[I 2024-09-13 00:30:56,051] A new study created in memory with name: RandomForestClassifier
[I 2024-09-13 00:30:57,196] Trial 0 finished with value: 0.8044207100956383 and parameters: {'n_estimators': 110, 'min_samples_leaf': 5, 'max_depth': 12}. Best is trial 0 with value: 0.8044207100956383.
[I 2024-09-13 00:30:58,103] Trial 1 finished with value: 0.8028055602792644 and parameters: {'n_estimators': 80, 'min_samples_leaf': 5, 'max_depth': 12}. Best is trial 0 with value: 0.8044207100956383.
[I 2024-09-13 00:30:58,896] Trial 2 finished with value: 0.7729045248570217 and parameters: {'n_estimators': 110, 'min_samples_leaf': 5, 'max_depth': 6}. Best is trial 0 with value: 0.8044207100956383.
[I 2024-09-13 00:30:59,884] Trial 3 finished with value: 0.7729045248570217 and parameters: {'n_estimators': 110, 'min_samples_leaf': 5, 'max_depth': 6}. Best is trial 0 with value: 0.8044207100956383.
[I 2024-09-13 00:31:01,551] Trial 4 finished with value: 0.7963789045354572 and parameters: {'n_est

In [35]:
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("F1-score на валидационном наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 200, 'min_samples_leaf': 5, 'max_depth': 15}
F1-score на валидационном наборе: 0.81


In [36]:
rf = ensemble.RandomForestClassifier(**study.best_params, random_state=42)

rf.fit(X_train, y_train)
y_test_pred = rf.predict(X_test)
print('F1-score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score на тестовом наборе: 0.79


# Выводы по оптимизации гиперпараметров моделей 

На основании проведённого подбора гиперпараметров для моделей 4 раными способами, наилучшее значение целевой метрики было достигнуто `Hyperopt` для модели случайного леса (`RandomForestClassifier`) — 0.80, cо следующими значениями параметров:

* `max_depth` (максимальная глубина 'дерева') — 12

* `min_samples_leaf` (минимальное количество объектов в 'листе') — 5

* `n_estimators` (количество 'деревьев в лесу') — 90.0

Таким образом модель с данным набором параметров одинаково хорошо отделяет классы друг от друга и обнаруживает класс 1 вообще, то есть находит большинство объектов, принадлежащих к классу 1.